In [1]:
%reload_ext autoreload
%autoreload 1

In [59]:
import os
import sys
import warnings
import logging
warnings.filterwarnings('ignore')

curruser = os.environ.get('USER')

# sys.path.insert(0, '/opt/workspace/{user}/system/support_library/'.format(user=curruser))
# sys.path.insert(0, '/opt/workspace/{user}/libs/'.format(user=curruser))
# sys.path.insert(0, '/opt/workspace/{user}/system/labdata/lib/'.format(user=curruser))

# sys.path.insert(0, '/opt/workspace/{}/notebooks/Clickstream_Analytics/scr'.format(curruser))
sys.path.insert(0, '/opt/workspace/{user}/notebooks/support_library/'.format(user=curruser)) 
sys.path.insert(0, '/opt/workspace/{user}/libs/python3.5/site-packages/'.format(user=curruser))
sys.path.insert(0, '/opt/workspace/{user}/notebooks/labdata/lib/'.format(user=curruser))

# sys.path.insert(0, '/home/{}/notebooks/support_library/'.format(curruser)) 
# sys.path.insert(0, '/home/{}/python35-libs/lib/python3.5/site-packages/'.format(curruser))
# sys.path.insert(0, '/home/{}/notebooks/labdata/lib/'.format(curruser))

# from tqdm import tqdm
# from tqdm._tqdm_notebook import tqdm_notebook
# tqdm_notebook.pandas()

from pathlib import Path
import pandas as pd
import numpy as np
import re 
import joblib
import subprocess

# pd.set_option('display.float_format', lambda x: '%.3f' % x)
# pd.set_option('display.max_colwidth', -1)
# pd.set_option('display.max_rows', None)
# pd.set_option('display.max_columns', None)

from spark_connector import SparkConnector
from sparkdb_loader import spark, log
# from spark_helper import SparkHelper
from connector import OracleDB
import pyspark
from pyspark import SparkContext, SparkConf, HiveContext
from pyspark.sql.functions import udf 
from pyspark.sql import functions as f
from pyspark.sql.window import Window
from pyspark.sql.types import *
from pyspark.sql.dataframe import DataFrame
from pyspark.ml import Pipeline, PipelineModel
from pyspark.ml.feature import StringIndexer

# import loader as load

# from corpora_process import utils
import time
from datetime import datetime
from dateutil.relativedelta import relativedelta

In [56]:
# src_path = os.getcwd() + '/../../src/'
# sys.path.insert(0, src_path) 

# from urlprocessing import ParseURL
# from collections import OrderedDict
# from CRMDictProds import *
from datetime import datetime

#----------------------------------------------------------------------------#
#----------------------------------------------------------------------------#

# logging.basicConfig(filename='./__upd_NRT_CLICKSTREAM__.log',
#                     level=logging.INFO,
#                     format='%(asctime)s %(levelname)s %(name)s %(message)s')
# logger = logging.getLogger(__name__)

#----------------------------------------------------------------------------#
#----------------------------------------------------------------------------#

In [4]:
def show(self, n=10):
    return self.limit(n).toPandas()

pyspark.sql.dataframe.DataFrame.show = show

In [5]:
%%time
sp = spark(schema='sklod_external_google_analytics',
           queue = 'ektov',
           process_label="NRT_SBBOL_",
           numofinstances=10, 
           numofcores=8,
           executor_memory='25g', 
           driver_memory='30g',
           dynamic_alloc=False,
           kerberos_auth=False
          ) 
print(sp.sc.version)
hive = sp.sql

2.4.0.cloudera2
CPU times: user 52.5 ms, sys: 25.4 ms, total: 77.9 ms
Wall time: 34.7 s


====================================================== 2022-03-31 ======================================================
# __init__ : begin


In [6]:
hive.sql("show tables in sklod_nrt_sbbol_clickstream").show()

,database,tableName,isTemporary
0,sklod_nrt_sbbol_clickstream,sbbol_events,False
1,sklod_nrt_sbbol_clickstream,sbbol_events_delta,False
2,sklod_nrt_sbbol_clickstream,test,False


## Work with parquets data natively

In [28]:
parq_id = '2022-03-30'
sdf = hive.read.option('dropFieldIfAllNull',True)\
               .parquet("hdfs://clsklod/data/core/nrt/sbbol_clickstream/pa/snp/sbbol-clickstream/event/timestampcolumn={}".format(parq_id))

sdf.registerTempTable("tmp_nrt_sbbol_part")

In [8]:
sdf.show()

,kafkaTimestampField,kafka_partition_offset,meta_apiKey,meta_sberId,meta_deviceVendor,meta_deviceModel,meta_deviceMemorySize,meta_operationSystem,meta_operationSystemVersion,meta_platform,...,data_value,data_timeStamp,data_geoLatitude,data_geoLongitude,data_cellularProvider,data_connectionType,data_batteryLevel,data_internalIP,data_tags,data_properties
0,2022-03-30T10:19:53.196+03:00,34_715983699,141f772ec05919125c66c5d9632eddd5a1faf901da4327...,,,,,,,WEB,...,,2022-03-30T10:19:51.883+03:00,,,,,,,[],"[(hitPageHostName, http://localhost:28016), (h..."
1,2022-03-30T10:19:53.202+03:00,34_715983700,141f772ec05919125c66c5d9632eddd5a1faf901da4327...,,,,,,,WEB,...,,2022-03-30T10:19:54.149+03:00,,,,,,,[],"[(hitPageHostName, https://sbi.sberbank.ru:944..."
2,2022-03-30T10:19:53.216+03:00,34_715983701,141f772ec05919125c66c5d9632eddd5a1faf901da4327...,,,,,,,WEB,...,,2022-03-30T10:21:15.875+03:00,,,,,,,[],"[(hitPageHostName, http://localhost:28016), (h..."
3,2022-03-30T10:19:53.219+03:00,34_715983702,141f772ec05919125c66c5d9632eddd5a1faf901da4327...,,,,,,,WEB,...,,2022-03-30T10:19:51.755+03:00,,,,,,,[],"[(hitPageHostName, http://localhost:28016), (h..."
4,2022-03-30T10:19:53.239+03:00,34_715983703,141f772ec05919125c66c5d9632eddd5a1faf901da4327...,,,,,,,WEB,...,,2022-03-30T16:20:50.104+09:00,,,,,,,[],"[(hitPageHostName, https://sbi.sberbank.ru:944..."
5,2022-03-30T10:19:53.240+03:00,34_715983704,141f772ec05919125c66c5d9632eddd5a1faf901da4327...,,,,,,,WEB,...,,2022-03-30T13:40:54.620+03:00,,,,,,,[],"[(hitPageHostName, https://sbi.sberbank.ru:944..."
6,2022-03-30T10:19:53.315+03:00,34_715983705,141f772ec05919125c66c5d9632eddd5a1faf901da4327...,,,,,,,WEB,...,,2022-03-30T09:19:52.602+02:00,,,,,,,[],"[(hitPageHostName, https://sbi.sberbank.ru:944..."
7,2022-03-30T10:19:53.333+03:00,34_715983706,141f772ec05919125c66c5d9632eddd5a1faf901da4327...,,,,,,,WEB,...,,2022-03-30T12:19:55.534+05:00,,,,,,,[],"[(hitPageHostName, http://localhost:28016), (h..."
8,2022-03-30T10:19:53.347+03:00,34_715983707,141f772ec05919125c66c5d9632eddd5a1faf901da4327...,,,,,,,WEB,...,,2022-03-30T10:19:52.070+03:00,,,,,,,[],"[(hitPageHostName, https://sbi.sberbank.ru:944..."
9,2022-03-30T10:19:53.356+03:00,34_715983708,141f772ec05919125c66c5d9632eddd5a1faf901da4327...,,,,,,,WEB,...,,2022-03-30T10:19:38.546+03:00,,,,,,,[],"[(hitPageHostName, https://sbi.sberbank.ru:944..."


In [30]:
qq=\
'''
select max(kafkaTimestampField) as max_ts, meta_apiKey, count(1) as cnt
from tmp_nrt_sbbol_part
group by meta_apiKey
'''

hive.sql(qq).collect()

[Row(max_ts='2022-03-30T23:59:59.880+03:00', meta_apiKey='400c6d7e0acb128da4b10d1579c034f9f92e48f43b4973374e86feb025d9408a', cnt=7376472),
 Row(max_ts='2022-03-30T23:59:59.852+03:00', meta_apiKey='f65912db7b21bfb64b793a83879d96199cf08242163f836b889136043e9e35f9', cnt=8533148),
 Row(max_ts='2022-03-30T23:53:56.470+03:00', meta_apiKey='', cnt=40137),
 Row(max_ts='2022-03-30T23:59:59.998+03:00', meta_apiKey='141f772ec05919125c66c5d9632eddd5a1faf901da43273099a8b2c9aba328d3', cnt=113985527),
 Row(max_ts='2022-03-30T22:34:35.598+03:00', meta_apiKey='1ca5959aaa4e22cef3c92413b3913923a5bf58450cef26798a451e1abd42fc1c', cnt=22333)]

In [31]:
hive.sql(qq).show()

,max_ts,meta_apiKey,cnt
0,2022-03-30T23:59:59.880+03:00,400c6d7e0acb128da4b10d1579c034f9f92e48f43b4973...,7376472
1,2022-03-30T23:59:59.852+03:00,f65912db7b21bfb64b793a83879d96199cf08242163f83...,8533148
2,2022-03-30T23:53:56.470+03:00,,40137
3,2022-03-30T23:59:59.998+03:00,141f772ec05919125c66c5d9632eddd5a1faf901da4327...,113985527
4,2022-03-30T22:34:35.598+03:00,1ca5959aaa4e22cef3c92413b3913923a5bf58450cef26...,22333


In [18]:
hive.sql('''select * from tmp_nrt_sbbol_part
            where from_unixtime(unix_timestamp(substring_index(kafkaTimestampField,'+',1), 
                                               "yyyy-MM-dd'T'HH:mm:ss.SSS")
                               )
                                >= timestamp('2022-03-30 14:00:00')
         '''
        ).show()

,kafkaTimestampField,kafka_partition_offset,meta_apiKey,meta_sberId,meta_deviceVendor,meta_deviceModel,meta_deviceMemorySize,meta_operationSystem,meta_operationSystemVersion,meta_platform,...,data_value,data_timeStamp,data_geoLatitude,data_geoLongitude,data_cellularProvider,data_connectionType,data_batteryLevel,data_internalIP,data_tags,data_properties
0,2022-03-30T14:20:17.106+03:00,34_717052829,141f772ec05919125c66c5d9632eddd5a1faf901da4327...,,,,,,,WEB,...,,2022-03-30T16:20:17.369+05:00,,,,,,,[],"[(hitPageHostName, https://sbi.sberbank.ru:944..."
1,2022-03-30T14:20:17.115+03:00,34_717052830,141f772ec05919125c66c5d9632eddd5a1faf901da4327...,,,,,,,WEB,...,,2022-03-30T16:20:16.652+05:00,,,,,,,[],"[(hitPageHostName, http://localhost:28016), (h..."
2,2022-03-30T14:20:17.149+03:00,34_717052831,141f772ec05919125c66c5d9632eddd5a1faf901da4327...,,,,,,,WEB,...,,2022-03-30T14:20:17.080+03:00,,,,,,,[],"[(hitPageHostName, http://localhost:28016), (h..."
3,2022-03-30T14:20:17.174+03:00,34_717052832,141f772ec05919125c66c5d9632eddd5a1faf901da4327...,,,,,,,WEB,...,,2022-03-30T14:19:45.606+03:00,,,,,,,[],"[(hitPageHostName, https://sbi.sberbank.ru:944..."
4,2022-03-30T14:20:17.180+03:00,34_717052833,141f772ec05919125c66c5d9632eddd5a1faf901da4327...,,,,,,,WEB,...,,2022-03-30T14:20:39.771+03:00,,,,,,,[],"[(hitPageHostName, http://localhost:28016), (h..."
5,2022-03-30T14:20:17.186+03:00,34_717052834,400c6d7e0acb128da4b10d1579c034f9f92e48f43b4973...,,,,,,,WEB,...,.....,2022-03-30T14:20:16.249+03:00,,,,WIFI,,,[],"[(hitPageHostName, https://www.sberbank.ru/ru/..."
6,2022-03-30T14:20:17.189+03:00,34_717052835,141f772ec05919125c66c5d9632eddd5a1faf901da4327...,,,,,,,WEB,...,,2022-03-30T14:20:18.133+03:00,,,,,,,[],"[(hitPageHostName, https://sbi.sberbank.ru:944..."
7,2022-03-30T14:20:17.240+03:00,34_717052836,141f772ec05919125c66c5d9632eddd5a1faf901da4327...,,,,,,,WEB,...,,2022-03-30T14:20:17.144+03:00,,,,,,,[],"[(hitPageHostName, http://localhost:28016), (h..."
8,2022-03-30T14:20:17.242+03:00,34_717052837,141f772ec05919125c66c5d9632eddd5a1faf901da4327...,,,,,,,WEB,...,,2022-03-30T14:20:15.868+03:00,,,,,,,[],"[(hitPageHostName, https://sbi.sberbank.ru:944..."
9,2022-03-30T14:20:17.264+03:00,34_717052838,141f772ec05919125c66c5d9632eddd5a1faf901da4327...,,,,,,,WEB,...,,2022-03-30T20:20:14.891+09:00,,,,,,,[],"[(hitPageHostName, https://sbi.sberbank.ru:944..."


In [19]:
hive.sql('''select
            max(from_unixtime(unix_timestamp(substring_index(kafkaTimestampField,'+',1), 
                                             "yyyy-MM-dd'T'HH:mm:ss.SSS")
                             )
               ) as max_kafkaTimestampField
           from tmp_nrt_sbbol_part
         '''
        ).show()

,max_kafkaTimestampField
0,2022-03-30 23:59:59


# Select interested partitions via HiveContext

In [36]:
## meta_apikey:
SITE_META = '400c6d7e0acb128da4b10d1579c034f9f92e48f43b4973374e86feb025d9408a'
SBBOL_META = '141f772ec05919125c66c5d9632eddd5a1faf901da43273099a8b2c9aba328d3'

## Find events from SITE channel

In [34]:
res = hive.sql('''select *
                  from sklod_nrt_sbbol_clickstream.sbbol_events 
                  where 
                  (timestamp(timestampcolumn) >= timestamp('2022-03-29')) and 
                  (timestamp(timestampcolumn) <= timestamp('2022-03-30')) and
                  meta_apikey = '{}'               
               '''.format(SITE_META))

In [12]:
res.cache()

DataFrame[kafkatimestampfield: string, kafka_partition_offset: string, meta_apikey: string, meta_sberid: string, meta_devicevendor: string, meta_devicemodel: string, meta_devicememorysize: string, meta_operationsystem: string, meta_operationsystemversion: string, meta_platform: string, meta_systemlanguage: string, meta_screensize: string, meta_browser: string, meta_timestamp: string, profile_deviceid: string, profile_sessionid: string, profile_partnerid: string, profile_appid: string, profile_sapid: string, profile_subid: string, profile_adid: string, profile_appversion: string, profile_applicationlanguage: string, profile_cookie: array<struct<key:string,value:string>>, profile_utm: array<struct<key:string,value:string>>, data_eventcategory: string, data_eventaction: string, data_eventtype: string, data_value: string, data_timestamp: string, data_geolatitude: string, data_geolongitude: string, data_cellularprovider: string, data_connectiontype: string, data_batterylevel: string, data_i

In [35]:
arr = res.take(3)[1].asDict()
arr

{'data_batterylevel': '',
 'data_cellularprovider': '',
 'data_connectiontype': 'WIFI',
 'data_eventaction': 'event',
 'data_eventcategory': 'SITE_Widget_VirtualAssistant',
 'data_eventtype': 'error',
 'data_geolatitude': '',
 'data_geolongitude': '',
 'data_internalip': '',
 'data_properties': [Row(key='hitPageHostName', value='https://www.sberbank.ru/ru/person/dist_services/inner_apps#about'),
  Row(key='eventLabel', value='.....'),
  Row(key='eventPageName', value='Мобильное приложение — СберБанк')],
 'data_tags': [],
 'data_timestamp': '2022-03-30T10:19:52.886+03:00',
 'data_value': '.....',
 'kafka_partition_offset': '34_715983746',
 'kafkatimestampfield': '2022-03-30T10:19:53.933+03:00',
 'meta_apikey': '400c6d7e0acb128da4b10d1579c034f9f92e48f43b4973374e86feb025d9408a',
 'meta_browser': 'Mozilla/5.0 (Linux; Android 11; Redmi Note 9 Pro) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.88 Mobile Safari/537.36',
 'meta_devicememorysize': '',
 'meta_devicemodel': '',
 'meta_d

## Add more attributes into consideration

In [42]:
res = hive.sql('''select 
                            timestampcolumn as sessiondate, 
                            data_eventCategory as eventCategory,
                            data_eventType as eventAction,
                            case when data_eventAction = "event" then "EVENT" 
                                 when data_eventAction = "pageview" then "PAGE"
                            else "EXCEPTION" end as hitType,
                            data_properties,
                            profile_cookie,
                            data_timestamp
                  from sklod_nrt_sbbol_clickstream.sbbol_events 
                  where 
                  (timestamp(timestampcolumn) >= timestamp('2022-03-30')) and 
                  (timestamp(timestampcolumn) <= timestamp('2022-03-31')) and
                  meta_apikey = '{}'
               '''.format(SITE_META)
              )

In [43]:
res.cache()

DataFrame[sessiondate: string, eventCategory: string, eventAction: string, hitType: string, data_properties: array<struct<key:string,value:string>>, profile_cookie: array<struct<key:string,value:string>>, data_timestamp: string]

In [44]:
res.count()

11844252

In [45]:
res.select(f.min("sessiondate")).show()

,min(sessiondate)
0,2022-03-30


## Data mapping: NRT --> GA

In [47]:
def get_value(col, key):
    if isinstance(col, str):
        js = json.loads(col)
        for item in js:
            if item['key'] == key:
                return item['value']
    elif isinstance(col, list):
        for i in range(len(col)):
            if col[i].key == key:
                return col[i].value
                    
@udf(IntegerType())
def get_timestamp(col):
    dt = datetime.strptime(col.split(".")[0], "%Y-%m-%dT%H:%M:%S")
    return int(dt.timestamp())

@udf(StringType())
def get_cid(col):
    try:
        return ".".join(get_value(col, "_ga").split(".")[-2:])
    except:
        return None

@udf(StringType())
def get_commonSegmentoUID(col):
    return get_value(col, "Segmento_UID")
        
@udf(StringType())
def get_sbbolUserId(col):
    return get_value(col, "sbbolUserId")    

@udf(StringType())
def get_hitPagePath(col):
    return get_value(col, "hitPagePath")


@udf(StringType())
def get_hitPageHostName(col):
    try:
        return get_value(col, "hitPageHostName").split(":")[1][2:]
    except:
        return None

@udf(StringType())
def get_eventLabel(col):
    return get_value(col, "eventLabel")

@udf(StringType())
def get_eventPageName(col):
    return get_value(col, "eventPageName")


In [48]:
res = res.withColumn("sessionStartTime", get_timestamp("data_timeStamp")) \
         .withColumn('hitPageHostName', get_hitPageHostName('data_properties')) \
         .withColumn('hitPagePath', get_hitPagePath('data_properties')) \
         .withColumn('sbbolUserId', get_sbbolUserId('data_properties')) \
         .withColumn("eventLabel", get_eventLabel('data_properties')) \
         .withColumn("eventPageName", get_eventPageName('data_properties')) \
         .withColumn("cid", get_cid('profile_cookie')) \
         .withColumn("commonSegmentoUID", get_commonSegmentoUID('profile_cookie')) \
         .drop('profile_cookie', 'data_properties', "data_timeStamp")
            
res = res.filter("(sbbolUserId is not Null) or (cid is not Null)")

# res = res.withColumn("visitNumber", f.lit(0)) \
#             .withColumn("hitPageTitle", f.lit(None).cast(stypes.StringType())) \
#             .withColumn("hitNumber", f.lit(0)) \
#             .withColumn("hitTime", f.lit(0)) \
#             .withColumn("ctl_loading", f.lit(None))

In [49]:
cols = \
['cid',
 'hitType',
 'hitPageHostName',
 'hitPagePath', 
 'eventCategory',
 'eventAction',
 'eventLabel',
 'eventPageName',
 'sessionStartTime',
 'sessiondate',
 'sbbolUserId',
 'commonSegmentoUID']
    
res = res.select(*cols)

res.filter("(eventAction = 'click')").show(40)

,cid,hitType,hitPageHostName,hitPagePath,eventCategory,eventAction,eventLabel,eventPageName,sessionStartTime,sessiondate,sbbolUserId,commonSegmentoUID
0,547191094.1648223911,EVENT,www.sberbank.ru/ru/search?searchid=2315434&tex...,None,SITE_Widget_YandexSearchResult,click,url_https:..yandex.ru.clck.jsredir,Поиск — СберБанк,1648716125,2022-03-31,None,
1,322422156.1569508742,EVENT,www.sberbank.ru/ru/s_m_business/search?searchi...,None,SITE_Widget_YandexSearchResult,click,url_https:..yandex.ru.clck.jsredir,Поиск — СберБанк,1648716127,2022-03-31,None,
2,587917039.1530630949,EVENT,www.sberbank.ru/ru/search?searchid=2315434&tex...,None,SITE_Widget_YandexSearchResult,click,url_https:..yandex.ru.clck.jsredir,Поиск — СберБанк,1648632063,2022-03-30,None,tU-H9thvr5JS
3,783368704.1648109164,EVENT,www.sberbank.ru/ru/search?searchid=2315434&tex...,None,SITE_Widget_YandexSearchResult,click,url_https:..yandex.ru.clck.jsredir,Поиск — СберБанк,1648624974,2022-03-30,None,
4,1621549813.1648023919,EVENT,www.sberbank.ru/ru/search?searchid=2321493&tex...,None,SITE_Widget_YandexSearchResult,click,url_https:..yandex.ru.clck.jsredir,Поиск — СберБанк,1648729293,2022-03-31,None,
5,304349856.1648729201,EVENT,www.sberbank.ru/ru/search?searchid=2315434&l10...,None,SITE_Widget_YandexSearchResult,click,url_https:..yandex.ru.clck.jsredir,Поиск — СберБанк,1648729378,2022-03-31,None,
6,728056775.1647348462,EVENT,www.sberbank.ru/ru/search?searchid=2315434&l10...,None,SITE_Widget_YandexSearchResult,click,url_https:..yandex.ru.clck.jsredir,Поиск — СберБанк,1648630257,2022-03-30,None,
7,1360324871.1633682467,EVENT,www.sberbank.ru/ru/search?searchid=2315434&l10...,None,SITE_Widget_YandexSearchResult,click,url_https:..yandex.ru.clck.jsredir,Поиск — СберБанк,1648630396,2022-03-30,None,
8,1405648944.1648627454,EVENT,www.sberbank.ru/ru/s_m_business/search?searchi...,None,SITE_Widget_YandexSearchResult,click,url_https:..yandex.ru.clck.jsredir,Поиск — СберБанк,1648720424,2022-03-31,None,
9,2047593676.1648720342,EVENT,www.sberbank.ru/ru/search?searchid=2315434&l10...,None,SITE_Widget_YandexSearchResult,click,url_https:..yandex.ru.clck.jsredir,Поиск — СберБанк,1648727614,2022-03-31,None,


## Obtain MAX timestamp in interested partition based on max_kafkatimestampfield attribute

In [26]:
ref_ts_str = '2022-03-30'
query = '''
           select 
                max(from_unixtime(unix_timestamp(substring_index(kafkaTimestampField,'+',1), 
                                                 "yyyy-MM-dd'T'HH:mm:ss.SSS")
                                 )
                   ) as max_kafkaTimestampField
           from sklod_nrt_sbbol_clickstream.sbbol_events
           where timestampcolumn = '{}' 
           and
           date_trunc('yyyy',
                      from_unixtime(unix_timestamp(substring_index(kafkaTimestampField,'+',1), 
                                    "yyyy-MM-dd'T'HH:mm:ss.SSS")  
                                   )
                      ) <= timestamp('2023-01-01')           
        '''.format(ref_ts_str)

ts = hive.sql(query)

In [27]:
%%time
ts.show()

CPU times: user 10.6 ms, sys: 3.39 ms, total: 14 ms
Wall time: 25.9 s


,max_kafkaTimestampField
0,2022-03-30 23:59:59


## Wrap all filters and steps into one UDF

In [51]:
NRT_SCHEMA = 'sklod_nrt_sbbol_clickstream'
NRT_EVENTS = 'sbbol_events'
REFDATE = '2022-03-31'

In [68]:
def get_df_sbbol(nrt_sbbol_clickstream=NRT_SCHEMA, nrt_schema=NRT_EVENTS, start_date=REFDATE, ref_ts = None):
    
    log("Formate table", sp.logger)
    
    if ref_ts is None:
        sql=\
        '''select               
                    timestampcolumn as sessiondate, 
                    data_eventCategory as eventCategory,
                    data_eventType as eventAction,
                    case when data_eventAction = "event" then "EVENT" 
                         when data_eventAction = "pageview" then "PAGE"
                    else "EXCEPTION" end as hitType,
                    data_properties,
                    profile_cookie,
                    meta_apikey,
                    from_unixtime(unix_timestamp(substring_index(kafkaTimestampField,'+',1), 
                                                 "yyyy-MM-dd'T'HH:mm:ss")) as sessionStartTime,
                    from_unixtime(unix_timestamp(substring_index(kafkaTimestampField,'+',1), 
                                                 "yyyy-MM-dd'T'HH:mm:ss"),"yyyy-MM-dd") as sessionDay                                                                          
           from {}.{} where timestamp(timestampcolumn) == timestamp('{}')
        '''.format(nrt_sbbol_clickstream, nrt_schema, start_date)
    else:
        sql=\
        '''select               
                    timestampcolumn as sessiondate, 
                    data_eventCategory as eventCategory,
                    data_eventType as eventAction,
                    case when data_eventAction = "event" then "EVENT" 
                         when data_eventAction = "pageview" then "PAGE"
                    else "EXCEPTION" end as hitType,
                    data_properties,
                    profile_cookie,
                    meta_apikey,
                    from_unixtime(unix_timestamp(substring_index(kafkaTimestampField,'+',1),
                                                 "yyyy-MM-dd'T'HH:mm:ss")) as sessionStartTime,
                    from_unixtime(unix_timestamp(substring_index(kafkaTimestampField,'.',1), 
                                                 "yyyy-MM-dd'T'HH:mm:ss"),"yyyy-MM-dd") as sessionDay                                                                          
           from {}.{} where (timestamp(timestampcolumn) >= timestamp('{}')) 
                      and (from_unixtime(unix_timestamp(substring_index(kafkaTimestampField,'+',1), 
                                                        "yyyy-MM-dd'T'HH:mm:ss")) > timestamp('{}'))
        '''.format(nrt_sbbol_clickstream, nrt_schema, start_date, ref_ts)        

    df_sbbol_table = hive.sql(sql)
    
# df_sbbol_table.withColumn("sessionStartTime", get_timestamp("data_timeStamp")) \    
    df_sbbol_table = \
    df_sbbol_table.withColumn('hitPageHostName', get_hitPageHostName('data_properties')) \
                  .withColumn('hitPagePath', get_hitPagePath('data_properties')) \
                  .withColumn('sbbolUserId', get_sbbolUserId('data_properties')) \
                  .withColumn("eventLabel", get_eventLabel('data_properties')) \
                  .withColumn("eventPageName", get_eventPageName('data_properties')) \
                  .withColumn("cid", get_cid('profile_cookie')) \
                  .withColumn("commonSegmentoUID", get_commonSegmentoUID('profile_cookie')) \
                  .drop('profile_cookie', 'data_properties')
    
    log("### Filtering fetched data", sp.logger)
    df_sbbol_table = df_sbbol_table.filter("(sbbolUserId is not Null) or (cid is not Null)")
    df_sbbol_table = df_sbbol_table.filter(f.col("sessiondate") == f.col("sessionDay")) 
    df_sbbol_table.cache()
        
#     print_and_log("### Removing records corresponding to 23:59:59 time")
#     max_sessiondate = df_sbbol_table.select(f.max("sessiondate").alias("max_sessiondate")).collect()[0]["max_sessiondate"]    
#     strange_datetime = max_sessiondate + ' ' + '23:59:59'
#     df_sbbol_table = df_sbbol_table.filter(f.col("sessionStartTime") != f.to_timestamp(f.lit(strange_datetime)))

    log("Calcuating count and max sessiondate", sp.logger)
    max_sessiondate = df_sbbol_table.select(f.max("sessiondate").alias("max_sessiondate")).collect()[0]["max_sessiondate"]    
    max_sessionStartTime = df_sbbol_table.select(f.max("sessionStartTime")\
                                                  .alias("max_sessionStartTime")).collect()[0]["max_sessionStartTime"]
    cnt = df_sbbol_table.count()
    
    return df_sbbol_table, max_sessiondate, max_sessionStartTime, cnt


In [69]:
# Get data and check for None
df_sbbol, maxSessionDate, max_sessionStartTime, cnt = get_df_sbbol(start_date=REFDATE, ref_ts = None)
# df_sbbol, maxSessionDate, max_sessionStartTime, cnt = get_df_sbbol(start_date=currSessionDate, ref_ts = ctl)

Formate table
### Filtering fetched data
Calcuating count and max sessiondate


In [70]:
df_sbbol.filter("meta_apikey ='{}'".format(SITE_META)).show()

,sessiondate,eventCategory,eventAction,hitType,meta_apikey,sessionStartTime,sessionDay,hitPageHostName,hitPagePath,sbbolUserId,eventLabel,eventPageName,cid,commonSegmentoUID
0,2022-03-31,SITE_Widget_VirtualAssistant,error,EVENT,400c6d7e0acb128da4b10d1579c034f9f92e48f43b4973...,2022-03-31 11:39:52,2022-03-31,www.sberbank.ru/ru/person/help/debet_cards_faq...,None,None,.....,Информация о продлении срока действия карт — С...,1611735171.1648715973,
1,2022-03-31,SITE_Widget_VirtualAssistant,error,EVENT,400c6d7e0acb128da4b10d1579c034f9f92e48f43b4973...,2022-03-31 11:39:52,2022-03-31,www.sberbank.ru/ru/person/credits/money/consum...,None,None,.....,Потребительский кредит наличными – оформить по...,196759654.1648715988,
2,2022-03-31,SITE_Widget_VirtualAssistant,error,EVENT,400c6d7e0acb128da4b10d1579c034f9f92e48f43b4973...,2022-03-31 11:39:52,2022-03-31,www.sberbank.ru/ru/person/credits/money/consum...,None,None,.....,Потребительский кредит наличными – оформить по...,196759654.1648715988,
3,2022-03-31,SITE_Widget_VirtualAssistant,error,EVENT,400c6d7e0acb128da4b10d1579c034f9f92e48f43b4973...,2022-03-31 11:39:52,2022-03-31,www.sberbank.ru/ru/person/help/debet_cards_faq...,None,None,.....,Информация о продлении срока действия карт — С...,1611735171.1648715973,
4,2022-03-31,SITE_Widget_OIB,zoom_bigger_vsp,EVENT,400c6d7e0acb128da4b10d1579c034f9f92e48f43b4973...,2022-03-31 11:39:52,2022-03-31,www.sberbank.ru/ru/oib?tab=vsp&nps30&nps31,None,None,Button Not button,"Офисы СберБанка, адреса ближайших офисов — Сбе...",712298855.1648715772,
5,2022-03-31,SITE_Widget_SiteHeader,change_region,EVENT,400c6d7e0acb128da4b10d1579c034f9f92e48f43b4973...,2022-03-31 11:39:52,2022-03-31,www.sberbank.ru/ru/person,None,None,from_undefined_to_77,СберБанк для физических лиц — банковские услуг...,2125238152.1648715909,
6,2022-03-31,SITE_Widget_VirtualAssistant,error,EVENT,400c6d7e0acb128da4b10d1579c034f9f92e48f43b4973...,2022-03-31 11:39:52,2022-03-31,www.sberbank.ru/ru/person/credits/money/consum...,None,None,.....,Потребительский кредит наличными – оформить по...,196759654.1648715988,
7,2022-03-31,SITE_Widget_RatesForm,select_iso_converter,EVENT,400c6d7e0acb128da4b10d1579c034f9f92e48f43b4973...,2022-03-31 11:39:52,2022-03-31,www.sberbank.ru/ru/quotes/currencies,None,None,selected_RUB__field_get,Курсы валют на сегодня онлайн: курс покупки и ...,518605027.1645712170,-ZGKUOlnLwWL
8,2022-03-31,SITE_Widget_DepositCatalog.ru.person.contribut...,calc_opened,EVENT,400c6d7e0acb128da4b10d1579c034f9f92e48f43b4973...,2022-03-31 11:39:52,2022-03-31,www.sberbank.ru/ru/person/contributions/deposi...,None,None,.....,Вклады в рублях с высоким процентом для физиче...,1291931607.1646407798,
9,2022-03-31,SITE_Widget_VirtualAssistant,open_app,EVENT,400c6d7e0acb128da4b10d1579c034f9f92e48f43b4973...,2022-03-31 11:39:51,2022-03-31,www.sberbank.ru/ru/person/help/debet_cards_faq...,None,None,2877acf9-aeff-4ce0-9fa8-7950cf7ab075,Информация о продлении срока действия карт — С...,1611735171.1648715973,
